# Introduction




## Requirements

## Data import

In [12]:
import subprocess

# Track if any packages are installed
installations_done = False

# Try to open 'requirements.txt' and read package names
try:
    with open('requirements.txt', 'r') as f:
        print("\nREQUIREMENTS:\n")
        requirements = f.read().splitlines()  # Read package names line by line
        print("\n".join(requirements))

        # For each package in requirements.txt, check if it is installed, and if not, install it
        for package in requirements:
            try:
                # Check if the package is installed by trying to import it
                __import__(package.split('==')[0])  # Get the package name before '==' if version is specified
                print(f"'{package}' is already installed.")
            except ImportError:
                # If the package is not installed, install it using pip
                print(f"'{package}' not found. Installing...")
                subprocess.check_call(['pip', 'install', package])
                installations_done = True  # Mark that an installation has been done

except FileNotFoundError:
    # If the file doesn't exist, generate it using pipreqsnb
    !pipreqsnb --savepath requirements.txt 02-IncomeData.ipynb
    print("'requirements.txt' generated successfully.")

    # Now read the contents of the newly generated file
    with open('requirements.txt', 'r') as f:
        print("\nREQUIREMENTS:\n")
        requirements = f.read().splitlines()
        print("\n".join(requirements))

        # Install the necessary packages
        for package in requirements:
            try:
                __import__(package.split('==')[0])
                print(f"'{package}' is already installed.")
            except ImportError:
                print(f"'{package}' not found. Installing...")
                subprocess.check_call(['pip', 'install', package])
                installations_done = True  # Mark that an installation has been done

# If any installations were done, print a completion message
if installations_done:
    print("\nInstallation of missing packages is complete.")
else:
    print("\nAll required packages were already installed.")


^C

REQUIREMENTS:

ucimlrepo
scikit-learn
'ucimlrepo' is already installed.
'scikit-learn' not found. Installing...

Installation of missing packages is complete.


In [13]:

from ucimlrepo import fetch_ucirepo
from sklearn.model_selection import train_test_split

# fetch dataset
adult = fetch_ucirepo(id=2)

# data (as pandas dataframes)
X = adult.data.features
y = adult.data.targets

# split into 80% training and 20% testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=3)

# metadata
print(adult.metadata)

# variable information
print(adult.variables)

# Check the size of the splits
print(f"Training set size: {X_train.shape}, Test set size: {X_test.shape}")



{'uci_id': 2, 'name': 'Adult', 'repository_url': 'https://archive.ics.uci.edu/dataset/2/adult', 'data_url': 'https://archive.ics.uci.edu/static/public/2/data.csv', 'abstract': 'Predict whether annual income of an individual exceeds $50K/yr based on census data. Also known as "Census Income" dataset. ', 'area': 'Social Science', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 48842, 'num_features': 14, 'feature_types': ['Categorical', 'Integer'], 'demographics': ['Age', 'Income', 'Education Level', 'Other', 'Race', 'Sex'], 'target_col': ['income'], 'index_col': None, 'has_missing_values': 'yes', 'missing_values_symbol': 'NaN', 'year_of_dataset_creation': 1996, 'last_updated': 'Tue Sep 24 2024', 'dataset_doi': '10.24432/C5XW20', 'creators': ['Barry Becker', 'Ronny Kohavi'], 'intro_paper': None, 'additional_info': {'summary': "Extraction was done by Barry Becker from the 1994 Census database.  A set of reasonably clean records was extracted using the fol

In [14]:
print(X_train)

       age         workclass  fnlwgt     education  education-num  \
45492   45           Private  165346  Some-college             10   
40885   32  Self-emp-not-inc   27207          11th              7   
35713   50           Private  134680       HS-grad              9   
26124   43         State-gov  101383       HS-grad              9   
15819   23           Private  353696       HS-grad              9   
...    ...               ...     ...           ...            ...   
25544   55           Private  208019       5th-6th              3   
48056   55         Local-gov  227386     Bachelors             13   
11513   47           Private  199058     Bachelors             13   
1688    53           Private  117496           9th              5   
5994    38           Private  239755       HS-grad              9   

           marital-status         occupation   relationship   race     sex  \
45492            Divorced    Exec-managerial      Unmarried  White  Female   
40885  Married-